In [ ]:
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from keras.preprocessing.image import ImageDataGenerator
from tensorflow import keras
from tensorflow.keras import layers, models, Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout, BatchNormalization, Activation, GlobalAveragePooling2D

In [ ]:
train_images = np.load("./Simpson/train.npy")
train_labels = np.load("./Simpson/train_label.npy")

In [ ]:
train_images = train_images/255
enc= OneHotEncoder()
train_labels_new = train_labels.reshape(-1,1)
enc.fit(train_labels_new)

#changed the names through one-hot encoding.
train_labels_onehot = np.array(enc.transform(train_labels_new).toarray())

print("one-hot encoding train shape is ",train_labels_onehot.shape)
print(train_labels_onehot)

one-hot encoding train shape is  (5210, 20)
[[1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ... 0. 0. 1.]]


In [ ]:
x_train, x_test, y_train, y_test = train_test_split(train_images, train_labels_onehot, test_size=0.2)
x_test, x_val, y_test, y_val = train_test_split(x_test, y_test, test_size=0.5)

In [ ]:
print(x_train.shape)
print(x_test.shape)
print(x_val.shape)

(4168, 64, 64, 3)
(521, 64, 64, 3)
(521, 64, 64, 3)


In [ ]:
class CellBlock(layers.Layer):
    def __init__(self, filter_num, stride=1):
        super(CellBlock, self).__init__()

        self.conv1 = Conv2D(filter_num, (3,3), strides=stride, padding='same')
        self.bn1 = BatchNormalization()
        self.relu = Activation('relu')

        self.conv2 = Conv2D(filter_num, (3,3), strides=1, padding='same')
        self.bn2 = BatchNormalization()

        if stride !=1:
            self.residual = Conv2D(filter_num, (1,1), strides=stride)
        else:
            self.residual = lambda x:x
        
    def call (self, inputs, training=None):

        x = self.conv1(inputs)
        x = self.bn1(x)
        x = self.relu(x)

        x = self.conv2(x)
        x = self.bn2(x)

        r = self.residual(inputs)

        x = layers.add([x, r])
        output = tf.nn.relu(x)

        return output

In [ ]:
class ResNet(models.Model):
    def __init__(self, layers_dims, nb_classes):
        super(ResNet, self).__init__()

        self.stem = Sequential([
            Conv2D(32, (7,7), strides=(2,2),padding='same'),
            BatchNormalization(),
            Activation('relu'),
        ]) 

        self.layer1 = self.build_cellblock(32, layers_dims[0], stride=1) 
        self.layer2 = self.build_cellblock(64, layers_dims[1], stride=2)
        self.layer3 = self.build_cellblock(128, layers_dims[2], stride=2)
        self.layer4 = self.build_cellblock(256, layers_dims[3], stride=2)

        self.avgpool = GlobalAveragePooling2D()
        self.fc = Dense(nb_classes, activation='softmax')
    
    def call(self, inputs, training=None):
        x=self.stem(inputs)

        x=self.layer1(x)
        x=self.layer2(x)
        x=self.layer3(x)
        x=self.layer4(x)
        x=self.avgpool(x)
        x=self.fc(x)

        return x

    def build_cellblock(self, filter_num, blocks, stride=1):
        res_blocks = Sequential()
        res_blocks.add(CellBlock(filter_num, stride))

        for _ in range(1, blocks):
            res_blocks.add(CellBlock(filter_num, stride=1))

        return res_blocks

In [ ]:
model = ResNet([2,2,2,2],20)
model.build(input_shape=(None, 64, 64, 3))
model.summary()

Model: "res_net"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
sequential (Sequential)      (None, 32, 32, 32)        4864      
_________________________________________________________________
sequential_1 (Sequential)    (None, 32, 32, 32)        37504     
_________________________________________________________________
sequential_2 (Sequential)    (None, 16, 16, 64)        132416    
_________________________________________________________________
sequential_3 (Sequential)    (None, 8, 8, 128)         526976    
_________________________________________________________________
sequential_4 (Sequential)    (None, 4, 4, 256)         2102528   
_________________________________________________________________
global_average_pooling2d (Gl multiple                  0         
_________________________________________________________________
dense (Dense)                multiple                  5140

In [ ]:
with tf.device('/gpu:0'):
  model.compile(optimizer='adam',
                loss='categorical_crossentropy',
                metrics=['accuracy'],
                )
  batch_size =32 #다음에는 32장으로
  model.fit(x_train, y_train, batch_size=batch_size, epochs=50, verbose=1, validation_data = (x_val,y_val))

Epoch 1/50
131/131 [==============================] - 38s 24ms/step - loss: 2.7912 - accuracy: 0.1854 - val_loss: 4.9931 - val_accuracy: 0.1036
Epoch 2/50
131/131 [==============================] - 2s 17ms/step - loss: 1.6669 - accuracy: 0.5063 - val_loss: 4.4218 - val_accuracy: 0.1766
Epoch 3/50
131/131 [==============================] - 2s 17ms/step - loss: 1.1337 - accuracy: 0.6589 - val_loss: 2.7624 - val_accuracy: 0.3743
Epoch 4/50
131/131 [==============================] - 2s 17ms/step - loss: 0.8821 - accuracy: 0.7341 - val_loss: 1.8222 - val_accuracy: 0.5547
Epoch 5/50
131/131 [==============================] - 2s 17ms/step - loss: 0.6020 - accuracy: 0.8136 - val_loss: 2.1618 - val_accuracy: 0.4818
Epoch 6/50
131/131 [==============================] - 2s 17ms/step - loss: 0.4038 - accuracy: 0.8794 - val_loss: 1.3782 - val_accuracy: 0.6641
Epoch 7/50
131/131 [==============================] - 2s 17ms/step - loss: 0.2799 - accuracy: 0.9219 - val_loss: 1.2296 - val_accuracy: 0.679

In [ ]:
train_datagen = ImageDataGenerator(
    rotation_range=40,
    zoom_range=0.2,
    width_shift_range=0,
    height_shift_range=0,
    shear_range=0.1,
    horizontal_flip=True)

val_datagen = ImageDataGenerator(
    rotation_range=40,
    zoom_range=0.2,
    width_shift_range=0,
    height_shift_range=0,
    shear_range=0.1,
    horizontal_flip=True)

train_generator = train_datagen.flow(x=x_train, y=y_train, batch_size=32, shuffle=True)
val_generator = val_datagen.flow(x=x_val, y=y_val, batch_size=32, shuffle=False)

model.fit_generator(train_generator, epochs=70, validation_data=val_generator)

Epoch 1/70
  2/131 [..............................] - ETA: 6s - loss: 0.0077 - accuracy: 1.0000

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


131/131 [==============================] - 5s 41ms/step - loss: 0.1134 - accuracy: 0.9669 - val_loss: 0.6459 - val_accuracy: 0.8752
Epoch 2/70
131/131 [==============================] - 5s 40ms/step - loss: 0.0705 - accuracy: 0.9782 - val_loss: 0.6945 - val_accuracy: 0.8580
Epoch 3/70
131/131 [==============================] - 5s 40ms/step - loss: 0.0708 - accuracy: 0.9796 - val_loss: 0.6702 - val_accuracy: 0.8733
Epoch 4/70
131/131 [==============================] - 5s 40ms/step - loss: 0.0620 - accuracy: 0.9796 - val_loss: 0.6204 - val_accuracy: 0.8733
Epoch 5/70
131/131 [==============================] - 5s 40ms/step - loss: 0.0496 - accuracy: 0.9849 - val_loss: 0.6552 - val_accuracy: 0.8848
Epoch 6/70
131/131 [==============================] - 5s 41ms/step - loss: 0.0394 - accuracy: 0.9880 - val_loss: 0.7647 - val_accuracy: 0.8618
Epoch 7/70
131/131 [==============================] - 5s 41ms/step - loss: 0.0676 - accuracy: 0.9798 - val_loss: 0.6749 - val_accuracy: 0.8829
Epoch 8/70

In [ ]:
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: 0.7123321890830994
Test accuracy: 0.8694817423820496


In [ ]:
import matplotlib.pyplot as plt

#saved_model이라는 이름으로 해당 모델을 저장한다.
model.save("./Simpson/saved_model")

In [ ]:
reconstructed_model = keras.models.load_model("./Simpson/saved_model")
with tf.device('/gpu:0'):
  score = reconstructed_model.evaluate(x_test, y_test, verbose=0)
  print('Test loss:', score[0])
  print('Test accuracy:', score[1])

Test loss: 0.05450688302516937
Test accuracy: 0.9904030561447144
